# Two one dimensional quantum wells 

[Go back to index](./index.ipynb)

In this widget, one can choice the position, height and width of two quantum wells. The widget gives the eigenfunctions and eigenvalues from the numerical solution. 

In [5]:
%matplotlib widget

from pylab import axes, setp, axis, xlabel, ylabel, title, plot, subplot, draw, connect, show
from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eig
import matplotlib.pyplot as plt


def doublewell_potential(x, x1, width1, height1, x2, width2, height2):
    xa = zeros(len(x))
    xb = zeros(len(x))
    
    for i in range(len(x)):
        if x[i] > x1 - width1/2.0 and x[i] < x1 + width1/2.0:
            xa[i] = height1
    
    for i in range(len(x)):
        if x[i] > x2 - width2/2.0 and x[i] < x2 + width2/2.0:
            xb[i] = height2
            
    return xa + xb


def diagonalisierung(hquer, L, N, pot = doublewell_potential, x1 = -0.1, width1 = 0.1, height1 = 0.0,
                     x2 = 0.1, width2 = 0.1, height2 = 0.0):
    """Berechne sortierte Eigenwerte und zugehoerige Eigenfunktionen.

       Eingabe:
         hquer: effektives hquer
         L: legt betrachtetes Intervall [-L,L] fest
         N: Zahl der Gitterpunkte, d.h. Groesse der Matrix
         pot: Potentialfunktion der Form pot(x, mu)
         mu: Potentialparameter
       Ausgabe:
         ew: sortierte Eigenwerte (Array der Laenge N)
         ef: entsprechend sortierte Eigenvektoren, ef[:,i] (Groesse N*N)
         x: Ortsgitter (Array der Laenge N)
         dx: Ortsgitterabstand
         V: Potential an den Stellen x (Array der Laenge N)
    """
    x = linspace(-L, L, N+2)[1:N+1]               # Gitterpunkte 
    dx = x[1] - x[0]                              # Gitterabstand
    V = pot(x, x1, width1, height1, x2, width2, height2)
    z = hquer**2 /2.0/dx**2                       # Nebendiagonalen
    h = (diag(V+2.0*z) + diag(-z*ones(N-1), -1)   # Aufstellen der
                      + diag(-z*ones(N-1), 1) )   #  Hamilton-Matrix

    ew, ef = eig(h)                               # Diagonalisierung
    ew = ew.real                                  # Realteil der Eigenwerte
    ind = argsort(ew)                             # Indizes f. sort. Array
    ew = ew[ind]                                  # Sortieren der ew nach ind
    ef = ef[:, ind]                               # Sortieren der Spalten von
                                                  #  ef nach ind
    ef = ef/sqrt(dx)                              # richtige Normierung 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.1, fak=0.009):
    """Plot der niedrigsten Eigenfunktionen 'ef' im Potential 'V(x)'
       auf Hoehe der Eigenwerte 'ew' in den Plotbereich 'ax'.
       
       Der optionale Parameter 'width' (mit Defaultwert 1)
       gibt die Linienstaerke beim Plot der Eigenfunktionen
       an. 'width' kann auch ein Array von Linienstaerken sein.
       'Emax' (mit Default-Wert V_0/10) legt die Energieobergrenze
       fuer den Plot fest.
       'fak' ist ein Skalierungsfaktor fuer die graphische Darstellung
       der Eigenfunktionen.
    """
    
    ax[0].plot(x, V, c='r')
    ax[1].plot(x, V, c='k', linewidth=1.0)
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V), Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V), Emax])
    
    ax[1].set_xlabel(r'$x/a$', fontsize = 10)
    ax[1].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
      
    indmax = sum(ew<=Emax)                       
    colors = ['b', 'g', 'r', 'c', 'm', 'y']      
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[1].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])


In [6]:
from ipywidgets import FloatSlider, jslink, VBox, HBox

mu = 0.06                                            # Potentialparameter
L = 1.5                                              # x-Bereich ist [-L,L]
N = 200                                              # Zahl der Gitterpunkte
hquer = 0.06                                         # effektives hquer
sigma_x = 0.1                                        # Breite Gauss
zeiten = linspace(0.0, 10.0, 400)                    # Zeiten f. Zeitentw.

style = {'description_width': 'initial'}

swidth1 = FloatSlider(value = 0.1, min = 0.1, max = 1.0, description = '1st well width: ', style = style)
sheight1 = FloatSlider(value = 0.0, min = 0.0, max = 5.0, description = 'Height: ')
sxpos1 = FloatSlider(value = -0.1, min = -L, max = L, description = r'$x_o$: ')

swidth2 = FloatSlider(value = 0.1, min = 0.1, max = 1.0, description = '2nd well width: ', style = style)
sheight2 = FloatSlider(value = 0.0, min = 0.0, max = 5.0, description = 'Height: ')
sxpos2 = FloatSlider(value = 0.1, min = -L, max = L, description = r'$x_o$: ')

ew, ef, x, dx, V = diagonalisierung(hquer, L, N)
    
fig, ax = plt.subplots(1, 2, figsize=(20, 8))

fig.suptitle('Numerial Solution of One Dimension Schroedinger Equation', fontsize = 10)
plot_eigenfunktionen(ax, ew, ef, x, V)


def on_width_change1(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        x1 = sxpos1.value, width1 = swidth1.value, height1 = sheight1.value,
                                        x2 = sxpos2.value, width2 = swidth2.value, height2 = sheight2.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)

def on_height_change1(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        x1 = sxpos1.value, width1 = swidth1.value, height1 = sheight1.value,
                                        x2 = sxpos2.value, width2 = swidth2.value, height2 = sheight2.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
def on_xpos_change1(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        x1 = sxpos1.value, width1 = swidth1.value, height1 = sheight1.value,
                                        x2 = sxpos2.value, width2 = swidth2.value, height2 = sheight2.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)

def on_width_change2(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        x1 = sxpos1.value, width1 = swidth1.value, height1 = sheight1.value,
                                        x2 = sxpos2.value, width2 = swidth2.value, height2 = sheight2.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)

def on_height_change2(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        x1 = sxpos1.value, width1 = swidth1.value, height1 = sheight1.value,
                                        x2 = sxpos2.value, width2 = swidth2.value, height2 = sheight2.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
def on_xpos_change2(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, 
                                        x1 = sxpos1.value, width1 = swidth1.value, height1 = sheight1.value,
                                        x2 = sxpos2.value, width2 = swidth2.value, height2 = sheight2.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
swidth1.observe(on_width_change1, names = 'value')
sheight1.observe(on_height_change1, names = 'value')
sxpos1.observe(on_xpos_change1, names = 'value')

swidth2.observe(on_width_change2, names = 'value')
sheight2.observe(on_height_change2, names = 'value')
sxpos2.observe(on_xpos_change2, names = 'value')

display(HBox([swidth1, sheight1, sxpos1]), HBox([swidth2, sheight2, sxpos2]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>